In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, window, current_timestamp, udf
from pyspark.sql.types import IntegerType

# Create SparkSession
spark = SparkSession.builder \
    .appName("MovingAverageStreamingFromFile") \
    .getOrCreate()

# Define a custom function to count vowels in a word
def count_vowels(word):
    vowels = "aeiouAEIOU"
    return sum(1 for letter in word if letter in vowels)

# Register the custom function as a UDF
count_vowels_udf = udf(count_vowels, IntegerType())

# Read streaming data from a file (initial path)
initial_path = "./data/"
lines = spark.readStream \
    .format("text") \
    .load(initial_path)

# Split the lines into words and add a timestamp
words = lines.select(explode(split(lines.value, " ")).alias("word"), current_timestamp().alias("timestamp"))

# Apply the custom function to count vowels in each word
words_with_vowel_count = words.withColumn("vowel_count", count_vowels_udf(words.word))

# Define a sliding window of 10 seconds with a slide duration of 5 seconds
'''
windowedAvgVowelCount = words_with_vowel_count \
    .groupBy(window(words_with_vowel_count.timestamp, "10 seconds", "5 seconds")) \
    .agg({"vowel_count": "avg"})
'''
windowedAvgVowelCount = words_with_vowel_count \
    .groupBy(window(words_with_vowel_count.timestamp, "10 seconds")) \
    .agg({"vowel_count": "avg"})


# Output the moving average of vowel counts to the console
query = windowedAvgVowelCount \
    .writeStream \
    .outputMode("update") \
    .trigger(processingTime='10 seconds') \
    .format("console") \
    .option("truncate", "false") \
    .start()

# Wait for the streaming to finish
query.awaitTermination()

# Stop the SparkSession
spark.stop()


24/04/09 12:15:22 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
24/04/09 12:15:24 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ba5d8d36-da0b-49af-86f4-7bd134c4e78d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/09 12:15:24 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+----------------+
|window                                    |avg(vowel_count)|
+------------------------------------------+----------------+
|{2024-04-09 12:15:20, 2024-04-09 12:15:30}|5.0             |
+------------------------------------------+----------------+



24/04/09 12:15:41 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 16990 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+----------------+
|window                                    |avg(vowel_count)|
+------------------------------------------+----------------+
|{2024-04-09 12:15:40, 2024-04-09 12:15:50}|5.0             |
+------------------------------------------+----------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+----------------+
|window                                    |avg(vowel_count)|
+------------------------------------------+----------------+
|{2024-04-09 12:15:50, 2024-04-09 12:16:00}|5.0             |
+------------------------------------------+----------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+----------------+
|window                                    |avg(vowel_count)|
+------------------------------------------+----------------+
|{2024-04-09 12:16:00, 2024-04-09 12:16:10}|5.0             |
+------------------------------------------+----------------+

